# Ajuste Fino de Modelos Open AI

Este notebook baseia-se nas orientações atuais fornecidas na documentação de [Ajuste Fino](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) da Open AI.

O ajuste fino melhora o desempenho dos modelos de base para a sua aplicação, ao reentreinar o modelo com dados e contexto adicionais relevantes para o caso de uso ou cenário específico. Note que técnicas de engenharia de prompts como _few shot learning_ e _retrieval augmented generation_ permitem-lhe enriquecer o prompt padrão com dados relevantes para melhorar a qualidade. No entanto, estas abordagens estão limitadas pelo tamanho máximo da janela de tokens do modelo de base escolhido.

Com o ajuste fino, estamos a reentreinar o próprio modelo com os dados necessários (permitindo usar muitos mais exemplos do que caberiam na janela máxima de tokens) – e a disponibilizar uma versão _personalizada_ do modelo que já não precisa de exemplos fornecidos no momento da inferência. Isto não só melhora a eficácia do nosso design de prompts (temos mais flexibilidade para usar a janela de tokens para outros fins), como também pode reduzir os custos (ao diminuir o número de tokens que precisamos de enviar para o modelo durante a inferência).

O ajuste fino tem 4 etapas:
1. Preparar os dados de treino e fazer o upload.
1. Executar o trabalho de treino para obter um modelo ajustado.
1. Avaliar o modelo ajustado e iterar para garantir a qualidade.
1. Disponibilizar o modelo ajustado para inferência quando estiver satisfeito.

Note que nem todos os modelos de base suportam ajuste fino – [consulte a documentação da OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para obter as informações mais recentes. Também é possível ajustar novamente um modelo que já foi ajustado anteriormente. Neste tutorial, vamos usar o `gpt-35-turbo` como o nosso modelo de base para ajuste fino.

---


### Passo 1.1: Preparar o Seu Conjunto de Dados

Vamos criar um chatbot que o ajuda a compreender a tabela periódica dos elementos, respondendo a perguntas sobre um elemento com uma quadra humorística. Neste tutorial simples, vamos apenas criar um conjunto de dados para treinar o modelo com alguns exemplos de respostas que mostram o formato esperado dos dados. Num caso real, seria necessário criar um conjunto de dados com muitos mais exemplos. Também pode ser possível utilizar um conjunto de dados aberto (para o seu domínio de aplicação), se existir, e adaptá-lo para ser usado no ajuste fino.

Como estamos a focar-nos no `gpt-35-turbo` e queremos uma resposta de turno único (chat completion), podemos criar exemplos usando [este formato sugerido](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) que reflete os requisitos de chat completion da OpenAI. Se espera conteúdo conversacional com vários turnos, deve usar o [formato de exemplo multi-turno](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), que inclui um parâmetro `weight` para indicar quais as mensagens que devem ser usadas (ou não) no processo de ajuste fino.

Vamos usar aqui o formato mais simples de turno único para o nosso tutorial. Os dados estão no [formato jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), com 1 registo por linha, cada um representado como um objeto em formato JSON. O excerto abaixo mostra 2 registos como exemplo – veja [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para o conjunto de exemplos completo (10 exemplos) que vamos usar no nosso tutorial de ajuste fino. **Nota:** Cada registo _deve_ ser definido numa única linha (não dividido por várias linhas como é habitual num ficheiro JSON formatado)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Num caso real, vai precisar de um conjunto de exemplos muito maior para obter bons resultados – o equilíbrio será entre a qualidade das respostas e o tempo/custos do ajuste fino. Estamos a usar um conjunto pequeno para podermos concluir o ajuste fino rapidamente e ilustrar o processo. Veja [este exemplo do OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para um tutorial de ajuste fino mais complexo.


### Passo 1.2 Carregar o Seu Conjunto de Dados

Carregue os dados utilizando a API de Ficheiros [como descrito aqui](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Note que, para executar este código, deve ter realizado previamente os seguintes passos:
 - Instalado o pacote Python `openai` (certifique-se de que utiliza uma versão >=0.28.0 para aceder às funcionalidades mais recentes)
 - Definido a variável de ambiente `OPENAI_API_KEY` com a sua chave de API da OpenAI
Para saber mais, consulte o [Guia de Configuração](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) disponibilizado para o curso.

Agora, execute o código para criar um ficheiro a partir do seu ficheiro JSONL local para carregamento.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Passo 2.1: Criar o trabalho de afinação com o SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Passo 2.2: Verificar o Estado do trabalho

Aqui estão algumas coisas que pode fazer com a API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Lista os últimos n trabalhos de afinação
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Obtém detalhes de um trabalho de afinação específico
- `client.fine_tuning.jobs.cancel(<job_id>)` - Cancela um trabalho de afinação
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Lista até n eventos do trabalho
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

O primeiro passo do processo é _validar o ficheiro de treino_ para garantir que os dados estão no formato correto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Passo 2.3: Registar eventos para acompanhar o progresso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Passo 2.4: Verificar o estado no Painel da OpenAI


Também pode consultar o estado visitando o site da OpenAI e explorando a secção _Fine-tuning_ da plataforma. Aqui poderá ver o estado do trabalho atual e acompanhar o histórico das execuções anteriores. Nesta captura de ecrã, pode ver que a execução anterior falhou e que a segunda tentativa foi bem-sucedida. Para contextualizar, isto aconteceu porque a primeira execução utilizou um ficheiro JSON com registos mal formatados – depois de corrigido, a segunda execução foi concluída com sucesso e o modelo ficou disponível para utilização.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.pt.png)


Também pode ver as mensagens de estado e métricas ao descer mais no painel visual, como mostrado:

| Mensagens | Métricas |
|:---|:---|
| ![Mensagens](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.pt.png) |  ![Métricas](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.pt.png)|


### Passo 3.1: Obter o ID e Testar o Modelo Ajustado no Código


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Passo 3.2: Carregar e Testar o Modelo Ajustado no Playground

Agora pode testar o modelo ajustado de duas formas. Primeiro, pode aceder ao Playground e usar o menu suspenso de Modelos para selecionar o seu novo modelo ajustado entre as opções disponíveis. A outra opção é utilizar a opção "Playground" apresentada no painel de Ajuste Fino (ver captura de ecrã acima), que abre a seguinte vista _comparativa_, mostrando lado a lado as versões do modelo base e do modelo ajustado para uma avaliação rápida.

![Estado do trabalho de ajuste fino](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.pt.png)

Basta preencher o contexto do sistema usado nos seus dados de treino e colocar a sua pergunta de teste. Vai reparar que ambos os lados são atualizados com o mesmo contexto e pergunta. Execute a comparação e verá a diferença nos resultados entre eles. _Repare como o modelo ajustado apresenta a resposta no formato que forneceu nos seus exemplos, enquanto o modelo base apenas segue o prompt do sistema_.

![Estado do trabalho de ajuste fino](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.pt.png)

Vai notar que a comparação também mostra o número de tokens para cada modelo e o tempo que demorou a inferência. **Este exemplo específico é bastante simples e serve apenas para ilustrar o processo, não refletindo um conjunto de dados ou cenário real**. Pode reparar que ambos os exemplos apresentam o mesmo número de tokens (o contexto do sistema e o prompt do utilizador são idênticos), sendo que o modelo ajustado demora mais tempo na inferência (modelo personalizado).

Em cenários reais, não irá usar um exemplo tão simples como este, mas sim ajustar o modelo com dados reais (por exemplo, um catálogo de produtos para apoio ao cliente), onde a qualidade da resposta será muito mais evidente. Nesse contexto, para obter uma qualidade de resposta equivalente com o modelo base, será necessário um maior trabalho de engenharia de prompts personalizados, o que aumentará o uso de tokens e, potencialmente, o tempo de processamento da inferência. _Para experimentar, consulte os exemplos de ajuste fino no OpenAI Cookbook para começar._



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, tenha em atenção que as traduções automáticas podem conter erros ou imprecisões. O documento original, na sua língua nativa, deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes da utilização desta tradução.
